# Carga de librerias

In [1]:
from pymongo import MongoClient
from json import dumps
import numpy as np
import os

client = MongoClient('localhost', 27017)
db = client[f"test_mongo"]

# Inserción de 1 millón de documentos en campos ya indexados

En esta sección se revisará la carga y el tamaño de almacenamiento de 1 millón de documentos indexados en la base de datos.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo.py -n 1

```

El script antes mencionado revise como entrada la cantidad de documentos que cargará a la base de datos. Secuencialmente, el código creará la nueva colección donde se cargarán los datos, creará los índices correspondientes a todos los campos y cargará la información almacenada en un documento JSON de forma masiva.

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_1M_documents_mongo.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos ha tomado un tiempo de **1 minuto y 1.51 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_1m``` de la siguiente manera.

In [116]:
cnt = db.extracto_cuenta_1m.count_documents({})
print(f"Actualmente existen {cnt} documentos en la colección 'extracto_cuenta_1m'")

Actualmente existen 1000000 documentos en la colección 'extracto_cuenta_1m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [124]:
file_sizes = []
n = 1

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data"))):
    file_stat = os.stat(os.path.join("../json-generated-data", json_file))
    file_sizes.append(file_stat)

    if i+1 == n:
        break

for file in file_sizes:
    print(f'Documento {json_file} con tamaño en memoria de {file.st_size} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 259734499 bytes


Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [5]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_1m",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_1m",
 "size": 206123814,
 "count": 1000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 103682048,
 "freeStorageSize": 409600,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none
...


In [ ]:
print("...")
print(r[109825:])

...
"indexBuilds": [],
 "totalIndexSize": 243048448,
 "totalSize": 346730496,
 "indexSizes": {
  "_id_": 11104256,
  "fecha_1": 41160704,
  "agencia_1": 6668288,
  "monto_1": 34279424,
  "saldo_1": 36990976,
  "descripcion_1": 6410240,
  "nota_1": 106434560
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento aproximado de 1 documento de 1 millón de registros con un peso en bruto de 247.70 mb (en escala base binario $2^{20}$), una vez comprimido en la base de datos es **98.87 mb** (en escala base binario $2^{20}$). Esto nos da un radio de compresión de 0.3991 y una reducción de tamaño en memoria del 60.08%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/storage_compass_1M_documents.png)

Mediante la herramienta gráfica compass se puede ver un reporte de 103.27 MB (en escala base decimal $10^{6}$) de tamaño de almacenamiento, lo cual convertido a escala binaria resulta en 98.23 mb, un número aproximado al obtenido mediante el script de python.

Otro número importante que se meustra en compass es el tamaño total de los índices (7 índices en este caso), siendo 243.05 MB (en escala base decimal $10^{6}$) o según los datos extraídos mediante python **231.79 mb** (en escala base binario $2^{20}$).

Una última métrica importante ha revisar es la suma del espacio asignado tanto a los documentos como a los índices en la colección. Es la suma de storageSize y indexSize. En el presente caso resultando un valor de **330.67 mb**.



| Tabla resumen de resultados              | Valor obtenido |
|------------------------------------------|----------------|
| Tamaño JSON bruto                        | 247.70 mb      |
| Tamaño de almacenamiento en mongoDB      | 98.87 mb       |
| Tamaño total de los índices              | 231.79 mb      |
| Tamaño total de almacenamiento colección | 330.67 mb      |

# Inserción de 10 millones de documentos en campos ya indexados

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documento

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-mongo.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-mongo.py -n 10

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/mongo-insert/load_10M_indexed_documents_mongo.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos ha tomado un tiempo de **3 horas 12 minutos y 37.32 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en la colección ```extracto_cuenta_1m``` de la siguiente manera.

In [112]:
cnt = db.extracto_cuenta_10m.count_documents({})
print(f"Actualmente existen {cnt} documentos en 'extracto_cuenta_10m'")

Actualmente existen 10000000 documentos en 'extracto_cuenta_10m'


## Tamaño de almacenamiento de datos

Para encontrar el tamaño de los archivos JSON en su forma bruta, antes de cargarse a la base de datos, se utilizará el siguiente script:

In [184]:
file_sizes = []
file_names = []

n = 10

for i, json_file in enumerate(sorted(os.listdir("../json-generated-data"))):
    file_stat = os.stat(os.path.join("../json-generated-data", json_file))
    file_sizes.append(file_stat)
    file_names.append(json_file)

    if i+1 == n:
        break

for file, f_name in zip(file_sizes, file_names):
    print(f'Documento {f_name} con tamaño en memoria de {file.st_size} bytes')

size_sum = np.sum(np.asarray(file_sizes))

print("")
print(f'Sumando un total de {size_sum} bytes')

Documento random-generated-data-1.json con tamaño en memoria de 259734499 bytes
Documento random-generated-data-10.json con tamaño en memoria de 259687666 bytes
Documento random-generated-data-11.json con tamaño en memoria de 259697531 bytes
Documento random-generated-data-12.json con tamaño en memoria de 259660386 bytes
Documento random-generated-data-13.json con tamaño en memoria de 259651170 bytes
Documento random-generated-data-14.json con tamaño en memoria de 259696780 bytes
Documento random-generated-data-15.json con tamaño en memoria de 259771941 bytes
Documento random-generated-data-16.json con tamaño en memoria de 259765264 bytes
Documento random-generated-data-17.json con tamaño en memoria de 259736398 bytes
Documento random-generated-data-18.json con tamaño en memoria de 259722533 bytes

Sumando un total de 52730502614 bytes


Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [ ]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_1m",
     "scale": 1
   }
), indent=1)

print(r[:300])
print("...")


{
 "ns": "test_mongo.extracto_cuenta_1m",
 "size": 206123814,
 "count": 1000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 103682048,
 "freeStorageSize": 409600,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none
...


In [ ]:
print("...")
print(r[109825:])

...
"indexBuilds": [],
 "totalIndexSize": 243048448,
 "totalSize": 346730496,
 "indexSizes": {
  "_id_": 11104256,
  "fecha_1": 41160704,
  "agencia_1": 6668288,
  "monto_1": 34279424,
  "saldo_1": 36990976,
  "descripcion_1": 6410240,
  "nota_1": 106434560
 },
 "scaleFactor": 1,
 "ok": 1.0
}


Para obtener las estadísticas de uso de disco por parte de la colección ```extracto_cuenta_10m``` en la base de datos ejecutaremos el siguiente comando:

In [115]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_10m",
     "scale": 1
   }
), indent=1)

print(r[:500])

{
 "ns": "test_mongo.extracto_cuenta_10m",
 "size": 2061015647,
 "count": 10000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 1043173376,
 "freeStorageSize": 2236416,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=snappy,cac


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 1 documento de 1 millón de registros con un peso en bruto de 259.73 mb (en escala base 10), **una vez comprimido en la base de datos es 103.27 mb (en escala base 10)**. Esto nos da un radio de compresión de 2.52 y una reducción de tamaño en memoria del 60.24%.

In [2]:
cnt = db.extracto_cuenta_30m.count_documents({})
print(f"Actualmente existen {cnt} documentos en 'extracto_cuenta_30m'")

Actualmente existen 30000000 documentos en 'extracto_cuenta_30m'


In [3]:
r = dumps(db.command(
   {
     "collStats": "extracto_cuenta_30m",
     "scale": 1
   }
), indent=1)

print(r[:500])

{
 "ns": "test_mongo.extracto_cuenta_30m",
 "size": 6182911222,
 "count": 30000000,
 "avgObjSize": 206,
 "numOrphanDocs": 0,
 "storageSize": 2984677376,
 "freeStorageSize": 954368,
 "capped": false,
 "wiredTiger": {
  "metadata": {
   "formatVersion": 1
  },
  "creationString": "access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=snappy,cach


In [4]:
print("...")
print(r[109825:])

...
llback to stable skipping stable rle": 0,
    "rollback to stable sweeping history store keys": 0,
    "rollback to stable updates removed from history store": 0,
    "transaction checkpoints due to obsolete pages": 0,
    "update conflicts": 0
   }
  }
 },
 "indexBuilds": [],
 "totalIndexSize": 3570143232,
 "totalSize": 6554820608,
 "indexSizes": {
  "_id_": 331038720,
  "fecha_1": 512319488,
  "agencia_1": 160526336,
  "monto_1": 468492288,
  "saldo_1": 493805568,
  "descripcion_1": 166879232,
  "nota_1": 1437081600
 },
 "scaleFactor": 1,
 "ok": 1.0
}
